In [2]:
from src.schemas.thesaurus.mads import SchemaMads
from datetime import datetime
from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [4]:
g = Graph()
g.parse('http://id.loc.gov/authorities/names/n2003042876.rdf')
g.serialize('Adichie.ttl')

<Graph identifier=Ncdefe5113c4a4efa8f4a39be3a83dc01 (<class 'rdflib.graph.Graph'>)>

In [3]:
req = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Authority",
                "label": "Authority"
            }
        },
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            }
        }
    ],
    "isMemberOfMADSCollection": [
        {
            "collection": {
                "value": "names"
            }
        }
    ],
    "elementList": [
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                "label": "FullNameElement"
            },
            "elementValue": "Adichie, Chimamanda Ngozi,"
        },
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                "label": "DateNameElement"
            },
            "elementValue": "1977-"
        }
    ],
    "authoritativeLabel": {
        "value": "Adichie, Chimamanda Ngozi, 1977-"
    },
    "birth": {
        "place": "Enugu (Nigeria)",
        "day": "15",
        "month": {
            "value": "09",
            "label": "Setembro"
        },
        "year": "1977"
    },
    "hasVariant": [
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Adichie, Amanda N."
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "(Amanda Ngozi),"
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1977-"
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "נגוזי אדיצ׳יה, צ׳יממנדה,"
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1977-"
                }
            ]
        }
    ],
    "hasAffiliation": [
        {
            "base": "loc",
            "label": "Drexel University",
            "uri": "http://id.loc.gov/authorities/names/n50075769"
        },
        {
            "base": "loc",
            "label": "Johns Hopkins University",
            "uri": "http://id.loc.gov/authorities/names/n79061226"
        },
        {
            "base": "loc",
            "label": "Eastern Connecticut State University",
            "uri": "http://id.loc.gov/authorities/names/nr99039944"
        },
        {
            "base": "loc",
            "label": "Princeton University",
            "uri": "http://id.loc.gov/authorities/names/n79055384"
        },
        {
            "base": "loc",
            "label": "Yale University",
            "uri": "http://id.loc.gov/authorities/names/n79043367"
        }
    ],
    "hasCloseExternalAuthority": [
        {
            "base": "www.wikidata.org",
            "label": "Chimamanda Ngozi Adichie",
            "uri": "http://www.wikidata.org/entity/Q230141"
        },
        {
            "base": "id.worldcat.org",
            "label": "Adichie, Chimamanda Ngozi, 1977-",
            "uri": "http://id.worldcat.org/fast/1911391"
        },
        {
            "base": "d-nb.info",
            "label": "Adichie, Chimamanda Ngozi",
            "uri": "https://d-nb.info/gnd/12975479X"
        }
    ],
    "identifiersLccn": "n2003042876"
}

In [4]:
request = SchemaMads(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = "10"

In [5]:
def HasAffiliationDoc(hasAffiliation, identifiedBy):
    affiliations = list()
    for i in hasAffiliation:
        obj  = {'id': f"authority#{identifiedBy}/hasAffiliation#{i.uri.split('/')[-1]}"}
        affiliation = obj | i.model_dump()
        affiliations.append(affiliation)
    return affiliations

def HasVariantDoc(hasVariant, doc):
    variants = list()
    hasVariants = list()
    for i in request.hasVariant:
        label = [j.elementValue for j in i.elementList]
        variantLabel = " ".join(label)
        variants.append(variantLabel)
        hasVariant = i.model_dump()
        hasVariant['variantLabel'] = variantLabel
        hasVariants.append(hasVariant)
    doc['variant'] = variants
    doc['hasVariant'] = hasVariants
    return doc


In [11]:
MDASRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")
BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
IDENTIFIERS = Namespace("http://id.loc.gov/vocabulary/identifiers/")
identifier = f"{settings.base_url}/authorities/{request.adminMetadata.identifiedBy}"
resource = URIRef(identifier)

g = Graph(identifier=identifier)
g.bind('madsrdf', MDASRDF)
g.bind('identifiers', IDENTIFIERS)
g.bind('bf', BF)

def HasAffiliation(g, resource, hasAffiliation, MDASRDF):
    for i in hasAffiliation:
        bNaffiliation = BNode()
        g.add((resource, MDASRDF.hasAffiliation, bNaffiliation))
        g.add((bNaffiliation, RDF.type, MDASRDF.Affiliation))
        uri = URIRef(i.uri)
        
        g.add((uri, RDF.type, MDASRDF.Organization))
        g.add((uri, MDASRDF.authoritativeLabel, Literal(i.label)))

    return g

HasAffiliation(g, resource, request.hasAffiliation, MDASRDF)

base='loc' label='Drexel University' uri='http://id.loc.gov/authorities/names/n50075769' affiliationStart=None affiliationEnd=None
base='loc' label='Johns Hopkins University' uri='http://id.loc.gov/authorities/names/n79061226' affiliationStart=None affiliationEnd=None
base='loc' label='Eastern Connecticut State University' uri='http://id.loc.gov/authorities/names/nr99039944' affiliationStart=None affiliationEnd=None
base='loc' label='Princeton University' uri='http://id.loc.gov/authorities/names/n79055384' affiliationStart=None affiliationEnd=None
base='loc' label='Yale University' uri='http://id.loc.gov/authorities/names/n79043367' affiliationStart=None affiliationEnd=None


In [7]:
from src.function.thesaurus.solr.birthDoc import BirthDoc
from src.function.thesaurus.solr.deathDoc import DeathDoc


def MakeDoc(request):
    authority = request.elementList[0].elementValue
    authority = authority.removesuffix(',')
    isMemberOfMADSCollection = [i.collection.value for i in request.isMemberOfMADSCollection]

    doc = { 
            'id': f'authority#{request.identifiersLocal}',
            'type': [i.type.label for i in request.resource],
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": request.authoritativeLabel,
            "authority": authority,
            "isMemberOfMADSCollection": isMemberOfMADSCollection
        }
    # Lang
    element = request.elementList[0]
    lang = element.elementLang
    if lang:
        doc['lang'] = lang.label

    if request.identifiersLccn:
        doc['identifiersLccn'] = request.identifiersLccn
        
    if request.imagem:
        doc['imagem'] = request.imagem

    if request.adminMetadata.changeDate:
        doc['changeDate'] = request.adminMetadata.changeDate.strftime("%Y-%m-%dT%H:%M:%S")
    
    if request.fullerName:
        doc['fullerName'] = request.fullerName
    
    metadados = ['birthDayDate', 'birthMonthDate','birthYearDate', 'birthDate', 'birthPlace', 'deathDate', 'deathPlace',
                 'deathDayDate', 'deathMonthDate', 'deathYearDate']
    for metadado in metadados:
        value = request.model_dump().get(metadado)
        if value:
            doc[metadado] = value

    if request.birth:
        doc = BirthDoc(doc, request.birth)

    if request.death:
        doc = DeathDoc(doc, request.death)
    
    # hasAffiliation  
    if request.hasAffiliation:
        affiliations = HasAffiliationDoc(request.hasAffiliation, request.adminMetadata.identifiedBy)
        doc['hasAffiliation'] = affiliations

    # hasVariant
    if request.hasVariant:
        doc = HasVariantDoc(request.hasVariant, doc)

    # hasCloseExternalAuthority
    if request.hasCloseExternalAuthority:
        uris = list()
        for i in request.hasCloseExternalAuthority:
            uri = {
                    'id': f"authority#{request.identifiersLocal}/hasCloseExternalAuthority#{i.uri.split('/')[-1]}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base }
            uris.append(uri)
        doc['hasCloseExternalAuthority'] = uris

    # Broader
    if request.hasBroaderAuthority:
        listMads = list()
        for i in request.hasBroaderAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasBroaderAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasBroaderAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasBroaderAuthority'] = listMads
        doc['hasBroaderAuthorityLabels']  = [i['label'] for i in listMads]

    # hasNarrowerAuthority
    if request.hasNarrowerAuthority:
        listMads = list()
        for i in request.hasNarrowerAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasNarrowerAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasNarrowerAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasNarrowerAuthority'] = listMads
        doc['hasNarrowerAuthorityLabels']  = [i['label'] for i in listMads]

    # hasReciprocalAuthority
    if request.hasReciprocalAuthority:
        listMads = list()
        for i in request.hasReciprocalAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasReciprocalAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasReciprocalAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasReciprocalAuthority'] = listMads
        doc['hasReciprocalAuthorityLabels']  = [i['label'] for i in listMads]
        
    # Occupation
    if request.occupation:
        occupations = list()
        for i in request.occupation:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/occupation#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/occupation#{i.label}",
                    'label': i.label,
                    'base': i.base }

            occupations.append(uri)
        doc['occupation'] = occupations
        doc['occupationLabels']  = [i['label'] for i in occupations]

    # fieldOfActivity
    if request.fieldOfActivity:
        fields = list()
        for i in request.fieldOfActivity:
            uri = {
                    'id': f"authority#{request.identifiersLocal}/fieldOfActivity#{i.uri.split('/')[-1]}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base }
            fields.append(uri)
        doc['fieldOfActivity'] = fields

    # identifiesRWO
    if request.identifiesRWO:
        fields = list()
        for i in request.identifiesRWO:
            identifier = i.uri.split("/")[-1]
            uri = {
                    'id': f"authority#{request.identifiersLocal}/identifiesRWO#{identifier}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base}
            fields.append(uri)
        doc['identifiesRWO'] = fields

    return doc


doc = MakeDoc(request)

{'typeVariant': {'value': 'http://www.loc.gov/mads/rdf/v1#PersonalName', 'label': 'PersonalName'}, 'elementList': [{'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': 'Adichie, Amanda N.', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': '(Amanda Ngozi),', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#DateNameElement', 'label': 'DateNameElement'}, 'elementValue': '1977-', 'elementLang': None}], 'variantLabel': 'Adichie, Amanda N. (Amanda Ngozi), 1977-'}
{'typeVariant': {'value': 'http://www.loc.gov/mads/rdf/v1#PersonalName', 'label': 'PersonalName'}, 'elementList': [{'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': 'נגוזי אדיצ׳יה, צ׳יממנדה,', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#D

In [8]:
doc

{'id': 'authority#None',
 'type': ['Authority', 'PersonalName'],
 'creationDate': '2024-05-02',
 'label': BfElement(value='Adichie, Chimamanda Ngozi, 1977-', metadata=None, label=None),
 'authority': 'Adichie, Chimamanda Ngozi',
 'isMemberOfMADSCollection': ['names'],
 'identifiersLccn': 'n2003042876',
 'birthDate': '1977-09-15',
 'birthPlace': 'Enugu (Nigeria)',
 'hasAffiliation': [{'id': 'authority#10/hasAffiliation#n50075769',
   'base': 'loc',
   'label': 'Drexel University',
   'uri': 'http://id.loc.gov/authorities/names/n50075769',
   'affiliationStart': None,
   'affiliationEnd': None},
  {'id': 'authority#10/hasAffiliation#n79061226',
   'base': 'loc',
   'label': 'Johns Hopkins University',
   'uri': 'http://id.loc.gov/authorities/names/n79061226',
   'affiliationStart': None,
   'affiliationEnd': None},
  {'id': 'authority#10/hasAffiliation#nr99039944',
   'base': 'loc',
   'label': 'Eastern Connecticut State University',
   'uri': 'http://id.loc.gov/authorities/names/nr99039

In [27]:
label = [j.elementValue for j in i.elementList]
label

['Adichie, Amanda N.', '(Amanda Ngozi),', '1977-']

In [12]:

    

HasVariantDoc(request.hasVariant)

{'typeVariant': {'value': 'http://www.loc.gov/mads/rdf/v1#PersonalName', 'label': 'PersonalName'}, 'elementList': [{'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': 'Adichie, Amanda N.', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': '(Amanda Ngozi),', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#DateNameElement', 'label': 'DateNameElement'}, 'elementValue': '1977-', 'elementLang': None}], 'variantLabel': 'Adichie, Amanda N. (Amanda Ngozi), 1977-'}
{'typeVariant': {'value': 'http://www.loc.gov/mads/rdf/v1#PersonalName', 'label': 'PersonalName'}, 'elementList': [{'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#FullNameElement', 'label': 'FullNameElement'}, 'elementValue': 'נגוזי אדיצ׳יה, צ׳יממנדה,', 'elementLang': None}, {'elementType': {'value': 'http://www.loc.gov/mads/rdf/v1#D